# Problem: Predicting Airplane Delays

The goals of this notebook are:
- Process and create a dataset from downloaded ZIP files
- Exploratory data analysis (EDA)
- Establish a baseline model and improve it

## Introduction to business scenario
You work for a travel booking website that is working to improve the customer experience for flights that were delayed. The company wants to create a feature to let customers know if the flight will be delayed due to weather when the customers are booking the flight to or from the busiest airports for domestic travel in the US. 

You are tasked with solving part of this problem by leveraging machine learning to identify whether the flight will be delayed due to weather. You have been given access to the a dataset of on-time performance of domestic flights operated by large air carriers. You can use this data to train a machine learning model to predict if the flight is going to be delayed for the busiest airports.

### Dataset
The provided dataset contains scheduled and actual departure and arrival times reported by certified US air carriers that account for at least 1 percent of domestic scheduled passenger revenues. The data was collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS). The dataset contains date, time, origin, destination, airline, distance, and delay status of flights for flights between 2014 and 2018.
The data are in 60 compressed files, where each file contains a CSV for the flight details in a month for the five years (from 2014 - 2018). The data can be downloaded from this [link](https://ucstaff-my.sharepoint.com/:f:/g/personal/ibrahim_radwan_canberra_edu_au/EhWeqeQsh-9Mr1fneZc9_0sBOBzEdXngvxFJtAlIa-eAgA?e=8ukWwa). Please download the data files and place them on a relative path. Dataset(s) used in this assignment were compiled by the Office of Airline Information, Bureau of Transportation Statistics (BTS), Airline On-Time Performance Data, available with the following [link](https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ). 

# Step 1: Prepare the environment 

Use one of the labs which we have practised on with the Amazon Sagemakers where you perform the following steps:
1. Start a lab.
2. Create a notebook instance and name it "oncloudproject".
3. Increase the used memory to 25 GB from the additional configurations.
4. Open Jupyter Lab and upload this notebook into it.
5. Upload the two combined CVS files (combined_csv_v1.csv and combined_csv_v2.csv), which you created in Part A of this project.

**Note:** In case of the data is too much to be uploaded to the AWS, please use 20% of the data only for this task.

# Step 2: Build and evaluate simple models

Write code to perform the follwoing steps:
1. Split data into training, validation and testing sets (70% - 15% - 15%).
2. Use linear learner estimator to build a classifcation model.
3. Host the model on another instance
4. Perform batch transform to evaluate the model on testing data
5. Report the performance metrics that you see better test the model performance 

Note: You are required to perform the above steps on the two combined datasets separatey and to comments on the difference.

In [4]:
import os, io, boto3, zipfile, requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sagemaker import get_execution_role, Session, image_uris
import sagemaker

In [12]:
bucket='c182567a4701745l12374482t1w921201583050-labbucket-jcugoasdssrh'

In [6]:
df = pd.read_csv('combined_csv_v1_20percent.csv')

# label column is named 'target' and is first column 
if 'target' not in df.columns:
    raise ValueError("Expected a 'target' column in the dataset.")
cols = df.columns.tolist()
cols = ['target'] + [c for c in cols if c != 'target']
df = df[cols]

# Optional: ensure target is 0/1 ints
df['target'] = pd.to_numeric(df['target'], errors='coerce').fillna(0).astype(int)

# Train/Val/Test split (70/15/15)
train, temp = train_test_split(
    df, test_size=0.30, random_state=42, stratify=df['target']
)
validate, test = train_test_split(
    temp, test_size=0.50, random_state=42, stratify=temp['target']
)

print(f"Train: {len(train)}  Validate: {len(validate)}  Test: {len(test)}")

Train: 228982  Validate: 49068  Test: 49068


In [13]:
prefix = 'lab3'                  # change if you like

s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    # header=False, index=False like your format
    dataframe.to_csv(csv_buffer, header=False, index=False)
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

# Filenames (mirroring your naming convention)
train_file    = 'flight_delay_train.csv'
validate_file = 'flight_delay_validate.csv'
test_file     = 'flight_delay_test.csv'

# Upload splits
upload_s3_csv(train_file, 'train', train)
upload_s3_csv(validate_file, 'validate', validate)
upload_s3_csv(test_file, 'test', test)

print(f"S3 paths:\n  s3://{bucket}/{prefix}/train/{train_file}\n  s3://{bucket}/{prefix}/validate/{validate_file}\n  s3://{bucket}/{prefix}/test/{test_file}")

S3 paths:
  s3://c182567a4701745l12374482t1w921201583050-labbucket-jcugoasdssrh/lab3/train/flight_delay_train.csv
  s3://c182567a4701745l12374482t1w921201583050-labbucket-jcugoasdssrh/lab3/validate/flight_delay_validate.csv
  s3://c182567a4701745l12374482t1w921201583050-labbucket-jcugoasdssrh/lab3/test/flight_delay_test.csv


In [14]:
region = boto3.Session().region_name
region

'us-east-1'

In [15]:
container = image_uris.retrieve('linear-learner', region)

In [16]:
s3_output_location="s3://{}/{}/output/".format(bucket,prefix)
# Create estimator
linear = sagemaker.estimator.Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

# Set hyperparameters for classification
linear.set_hyperparameters(
    predictor_type='binary_classifier',
    mini_batch_size=200,
    epochs=10,
    loss='logistic',
    normalize_data=True
)

In [17]:
train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

linear.fit(inputs=data_channels, logs=False)

print('ready for hosting!')

INFO:sagemaker:Creating training-job with name: linear-learner-2025-10-30-04-22-09-606



2025-10-30 04:22:12 Starting - Starting the training job..
2025-10-30 04:22:27 Starting - Preparing the instances for training...
2025-10-30 04:22:49 Downloading - Downloading input data.........
2025-10-30 04:23:40 Downloading - Downloading the training image...............
2025-10-30 04:25:01 Training - Training image download completed. Training in progress........................................................................................................
2025-10-30 04:33:46 Uploading - Uploading generated training model...
2025-10-30 04:34:04 Completed - Training job completed
ready for hosting!


In [18]:
predictor = linear.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

INFO:sagemaker:Creating model with name: linear-learner-2025-10-30-04-34-11-141
INFO:sagemaker:Creating endpoint-config with name linear-learner-2025-10-30-04-34-11-141
INFO:sagemaker:Creating endpoint with name linear-learner-2025-10-30-04-34-11-141


-------!

In [20]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import json, numpy as np

# Make sure features are numeric float32
X_test = test.drop('target', axis=1).astype('float32')
y_test = test['target']

# Configure endpoint I/O
predictor.serializer   = CSVSerializer()        # send CSV
predictor.deserializer = JSONDeserializer()     # receive JSON
predictor.content_type = 'text/csv'
predictor.accept       = 'application/json'

pred_labels, pred_scores = [], []
batch_size = 200

for start in range(0, len(X_test), batch_size):
    batch = X_test.iloc[start:start+batch_size]
    payload = batch.to_csv(header=False, index=False).strip()  # no trailing newline

    resp = predictor.predict(payload)          # <-- now a dict
    preds = resp['predictions']                # list of dicts

    for p in preds:
        pred_labels.append(int(p['predicted_label']))
        pred_scores.append(float(p['score']))

pred_labels = np.array(pred_labels)
pred_scores = np.array(pred_scores)


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score
acc  = accuracy_score(y_test, pred_labels)
prec = precision_score(y_test, pred_labels, zero_division=0)
rec  = recall_score(y_test, pred_labels, zero_division=0)
f1   = f1_score(y_test, pred_labels, zero_division=0)
cm   = confusion_matrix(y_test, pred_labels)

print(f"Accuracy    : {acc:.4f}")
print(f"Precision   : {prec:.4f}")
print(f"Recall      : {rec:.4f}")
print(f"F1-score    : {f1:.4f}")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, pred_labels, zero_division=0))

Accuracy    : 0.7915
Precision   : 0.4615
Recall      : 0.0012
F1-score    : 0.0023
Confusion Matrix:
 [[38827    14]
 [10215    12]]

Classification Report:
               precision    recall  f1-score   support

           0       0.79      1.00      0.88     38841
           1       0.46      0.00      0.00     10227

    accuracy                           0.79     49068
   macro avg       0.63      0.50      0.44     49068
weighted avg       0.72      0.79      0.70     49068



In [23]:
df = pd.read_csv('combined_csv_v2_20percent.csv')

# label column is named 'target' and is first column 
if 'target' not in df.columns:
    raise ValueError("Expected a 'target' column in the dataset.")
cols = df.columns.tolist()
cols = ['target'] + [c for c in cols if c != 'target']
df = df[cols]

# Optional: ensure target is 0/1 ints
df['target'] = pd.to_numeric(df['target'], errors='coerce').fillna(0).astype(int)

# Train/Val/Test split (70/15/15)
train, temp = train_test_split(
    df, test_size=0.30, random_state=42, stratify=df['target']
)
validate, test = train_test_split(
    temp, test_size=0.50, random_state=42, stratify=temp['target']
)

print(f"Train: {len(train)}  Validate: {len(validate)}  Test: {len(test)}")

Train: 228982  Validate: 49068  Test: 49068


In [24]:
# Upload splits
upload_s3_csv(train_file, 'train', train)
upload_s3_csv(validate_file, 'validate', validate)
upload_s3_csv(test_file, 'test', test)

print(f"S3 paths:\n  s3://{bucket}/{prefix}/train/{train_file}\n  s3://{bucket}/{prefix}/validate/{validate_file}\n  s3://{bucket}/{prefix}/test/{test_file}")

S3 paths:
  s3://c182567a4701745l12374482t1w921201583050-labbucket-jcugoasdssrh/lab3/train/flight_delay_train.csv
  s3://c182567a4701745l12374482t1w921201583050-labbucket-jcugoasdssrh/lab3/validate/flight_delay_validate.csv
  s3://c182567a4701745l12374482t1w921201583050-labbucket-jcugoasdssrh/lab3/test/flight_delay_test.csv


In [25]:
train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

linear.fit(inputs=data_channels, logs=False)

print('ready for hosting!')

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: linear-learner-2025-10-30-04-41-50-322



2025-10-30 04:41:51 Starting - Starting the training job..
2025-10-30 04:42:05 Starting - Preparing the instances for training...
2025-10-30 04:42:29 Downloading - Downloading input data.........
2025-10-30 04:43:19 Downloading - Downloading the training image.............
2025-10-30 04:44:30 Training - Training image download completed. Training in progress..........................................................................................
2025-10-30 04:52:02 Uploading - Uploading generated training model..
2025-10-30 04:52:20 Completed - Training job completed
ready for hosting!


In [31]:
X_test = test.drop('target', axis=1).astype('float32')
y_test = test['target']

In [33]:
y_test.head()

234564    0
32337     0
10046     1
228703    0
101846    0
Name: target, dtype: int64

In [35]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import json, numpy as np

# Make sure features are numeric float32
X_test = test.drop('target', axis=1).astype('float32')
y_test = test['target']

# Configure endpoint I/O
predictor.serializer   = CSVSerializer()        # send CSV
predictor.deserializer = JSONDeserializer()     # receive JSON
predictor.content_type = 'text/csv'
predictor.accept       = 'application/json'

pred_labels, pred_scores = [], []
batch_size = 200

for start in range(0, len(X_test), batch_size):
    batch = X_test.iloc[start:start+batch_size]
    payload = batch.to_csv(header=False, index=False).strip()  # no trailing newline

    resp = predictor.predict(payload)          # <-- now a dict
    preds = resp['predictions']                # list of dicts

    for p in preds:
        pred_labels.append(int(p['predicted_label']))
        pred_scores.append(float(p['score']))

pred_labels = np.array(pred_labels)
pred_scores = np.array(pred_scores)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:22                                                                                   │
│                                                                                                  │
│   19 │   batch = X_test.iloc[start:start+batch_size]                                             │
│   20 │   payload = batch.to_csv(header=False, index=False).strip()  # no trailing newline        │
│   21 │                                                                                           │
│ ❱ 22 │   resp = predictor.predict(payload)          # <-- now a dict                             │
│   23 │   preds = resp['predictions']                # list of dicts                              │
│   24 │                                                                                           │
│   25 │   for p in preds:                                                                         │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_predictor.py:2 │
│ 12 in predict                                                                                    │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:602 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    605                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)         

In [36]:
batch_X = X_test.iloc[:,1:];
batch_X.head()

,DepHourofDay,AWND_O,PRCP_O,TAVG_O,AWND_D,PRCP_D,TAVG_D,SNOW_O,SNOW_D,Year_2015,...,Origin_SFO,Dest_CLT,Dest_DEN,Dest_DFW,Dest_IAH,Dest_LAX,Dest_ORD,Dest_PHX,Dest_SFO,is_holiday_True
234564,12.0,43.0,0.0,-69.0,18.0,0.0,157.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32337,14.0,33.0,0.0,115.0,51.0,0.0,214.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10046,12.0,34.0,0.0,186.0,40.0,0.0,197.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
228703,9.0,36.0,0.0,194.0,50.0,0.0,284.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101846,21.0,42.0,0.0,171.0,64.0,0.0,192.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

In [ ]:
batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

linear_transformer = linear.transformer(
    instance_count=1,
    instance_type='ml.m4.xlarge',   # You can change to ml.t2.medium if smaller is enough
    strategy='MultiRecord',
    assemble_with='Line',
    output_path=batch_output
)

# Run the transform job
linear_transformer.transform(
    data=batch_input,
    data_type='S3Prefix',
    content_type='text/csv',
    split_type='Line'
)

# Wait for completion
linear_transformer.wait()

INFO:sagemaker:Creating model with name: linear-learner-2025-10-30-05-17-22-198
INFO:sagemaker:Creating transform job with name: linear-learner-2025-10-30-05-17-22-900


..

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),sep=',',names=['class'])
target_predicted.head(5)

In [29]:
acc  = accuracy_score(y_test, pred_labels)
prec = precision_score(y_test, pred_labels, zero_division=0)
rec  = recall_score(y_test, pred_labels, zero_division=0)
#f1   = f1_score(y_test, pred_labels, zero_division=0)
cm   = confusion_matrix(y_test, pred_labels)

print(f"Accuracy    : {acc:.4f}")
print(f"Precision   : {prec:.4f}")
print(f"Recall      : {rec:.4f}")
#print(f"F1-score    : {f1:.4f}")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, pred_labels, zero_division=0))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 acc  = accuracy_score(y_test, pred_labels)                                                  │
│    2 prec = precision_score(y_test, pred_labels, zero_division=0)                                │
│    3 rec  = recall_score(y_test, pred_labels, zero_division=0)                                   │
│    4 #f1   = f1_score(y_test, pred_labels, zero_division=0)                                      │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/_param_validati │
│ on.py:218 in wrapper                                                                             │
│                                                                                                  │
│   215 │   │   │   │   │   │   prefer_skip_nested_validation or global_skip_validation            │
│   216 │   │   │   │   │   )                                                                      │
│   217 │   │   │   │   ):                                                                         │
│ ❱ 218 │   │   │   │   │   return func(*args, **kwargs)                                           │
│   219 │   │   │   except InvalidParameterError as e:                                             │
│   220 │   │   │   │   # When the function is just a wrapper around an estimator, we allow        │
│   221 │   │   │   │   # the function to delegate validation to the estimator, but we replace     │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classificati │
│ on.py:359 in accuracy_score                                                                      │
│                                                                                                  │
│    356 │   xp, _, device = get_namespace_and_device(y_true, y_pred, sample_weight)               │
│    357 │   # Compute accuracy for each possible representation                                   │
│    358 │   y_true, y_pred = attach_unique(y_true, y_pred)                                        │
│ ❱  359 │   y_type, y_true, y_pred = _check_targets(y_true, y_pred)                               │
│    360 │   check_consistent_length(y_true, y_pred, sample_weight)                                │
│    361 │                                                                                         │
│    362 │   if y_type.startswith("multilabel"):                                                   │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classificati │
│ on.py:97 in _check_targets                                                                       │
│                                                                                                  │
│     94 │   y_pred : array or indicator matrix                                                    │
│     95 │   """                                                                                   │
│     96 │   xp, _ = get_namespace(y_true, y_pred)                                                 │
│ ❱   97 │   check_consistent_length(y_true, y_pred)                                               │
│     98 │   type_true = type_of_target(y_true, input_name="y_true")                               │
│     99 │   type_pred = type_of_target(y_pred, input_name="y_pred")                               │
│    100                                                     

Probe payload (first 120 chars): 602.0,12.0,43.0,0.0,-69.0,18.0,0.0,157.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Single-row predict still failing: ModelError('An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "unable to evaluate payload provided". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/linear-learner-2025-10-30-04-34-11-141 in account 921201583050 for more information.')
Raw runtime call also failed: ModelError('An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "unable to evaluate payload provided". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/linear-learner-2025-10-30-04-34-11-141 in account 921201583050 for more information.')


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:82                                                                                   │
│                                                                                                  │
│    79                                                                                            │
│    80 # ---------- 5) Single-row smoke test ----------                                           │
│    81 try:                                                                                       │
│ ❱  82 │   _lbl, _sc = predict_batch_json_then_csv(X_test.iloc[[0]])                              │
│    83 │   print("Single-row prediction OK:", _lbl, _sc)                                          │
│    84 except Exception as ex:                                                                    │
│    85 │   print("Single-row predict still failing:", repr(ex))                                   │
│                                                                                                  │
│ in predict_batch_json_then_csv:61                                                                │
│                                                                                                  │
│    58 │   # fallback: ask for CSV output                                                         │
│    59 │   predictor.deserializer = CSVDeserializer()                                             │
│    60 │   predictor.accept = 'text/csv'                                                          │
│ ❱  61 │   resp = predictor.predict(payload)                                                      │
│    62 │   if isinstance(resp, (bytes, bytearray)):                                               │
│    63 │   │   resp = resp.decode('utf-8', errors='ignore')                                       │
│    64 │   lines = [ln.strip() for ln in str(resp).strip().splitlines() if ln.strip()]            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_predictor.py:2 │
│ 12 in predict                                                                                    │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:602 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                               

# Step 3: Build and evaluate ensembe models

Write code to perform the follwoing steps:
1. Split data into training, validation and testing sets (70% - 15% - 15%).
2. Use xgboost estimator to build a classifcation model.
3. Host the model on another instance
4. Perform batch transform to evaluate the model on testing data
5. Report the performance metrics that you see better test the model performance 
6. write down your observation on the difference between the performance of using the simple and ensemble models.
Note: You are required to perform the above steps on the two combined datasets separatey.

In [4]:
# Write the code here and add cells as you need

In [5]:
# Write the final comments here and turn the cell type into markdown